In [2]:
import pandas as pd
import numpy as np
import progressbar

In [3]:
df = pd.read_csv("fer2013.csv")

In [4]:
df.shape

(35887, 3)

In [5]:
f = open("fer2013.csv")

In [6]:
f.__next__()

'emotion,pixels,Usage\n'

In [7]:
trainImage , trainLabels=[],[]

In [8]:
valImage, valLabels = [] ,[]

In [9]:
testImage , testLabels = [] ,[]

In [10]:
widgets = ["Image Processing" , progressbar.Percentage()," ", progressbar.Bar(), " " , progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval= 35887 , widgets = widgets).start()

for n ,i in enumerate(f):
    lable , image , uses = i.strip().split(",")
    image = np.array(image.split(" "),dtype = "uint8")
    image = image.reshape((48,48,1))
    if uses == "Training":
        trainImage.append(image)
        trainLabels.append(lable)
    elif uses == "PrivateTest":
        valImage.append(image)
        valLabels.append(lable)
    else:
        testImage.append(image)
        testLabels.append(lable)
    pbar.update(n)
pbar.finish()


        

Image Processing100% |##########################################| Time: 0:00:17


In [11]:
trainImage = np.array(trainImage)
valImage = np.array(valImage)
testImage = np.array(testImage)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Flatten , Dense , Dropout

import warnings
warnings.filterwarnings('ignore')

In [19]:
from tensorflow.keras.preprocessing.image import img_to_array , load_img , ImageDataGenerator

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from imutils import paths
import numpy as np
import cv2
import os
import progressbar

In [21]:
def LeNet(width , height , depth , classes):
    model = Sequential()
    
    model.add(Conv2D(20, (5,5) , activation = "relu" , input_shape = (width, height , depth)))
    model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))
    
    model.add(Conv2D(50, (5,5), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Flatten())
    model.add(Dense(classes , activation = "softmax"))
    return model

In [22]:
augmentation = ImageDataGenerator(rotation_range=30 ,width_shift_range=0.1, height_shift_range=0.1 , zoom_range=0.2 , horizontal_flip=True , fill_mode="nearest")

In [23]:
from sklearn.preprocessing import LabelBinarizer

trainLabels = LabelBinarizer().fit_transform(trainLabels)
valLabels = LabelBinarizer().fit_transform(valLabels)
testLabels = LabelBinarizer().fit_transform(testLabels)

In [24]:

model = LeNet(48, 48, 1, len(trainLabels[0]))
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print("Model is Going to Train.....")
#H = model.fit_generator(augmentation.flow(trainImage, trainLabels, batch_size = 32), validation_data=(valImage, valLabels), steps_per_epoch=len(trainImage)//32, epochs=10, verbose=1)             


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model is Going to Train.....


In [25]:
### Live Emotion Prediction

In [26]:
Emotion = ["Angry" , "Disgust" , "Fear" , "Happy" , "Sad", "Surprise" , "Neutral" ]

In [27]:
from tensorflow.keras.models import load_model

In [28]:
from tensorflow.keras.preprocessing.image import img_to_array , load_img


In [29]:
loaded_model = load_model("facial_emaotion_model.hdf5")

In [ ]:
import cv2
face_class = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)
while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_class.detectMultiScale(gray, 1.3, 5)

    for x,y,w,h in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (64,64))
        roi = roi.astype("float")/255
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)
        
        pred = loaded_model.predict(roi)[0]
        prob = np.max(pred)
        label = Emotion[pred.argmax()]
        Status = label + "  " + str(prob)
        color = ""
        if label == "Happy":
            color = (0,255,0)
        else:
            color = (0,0,255)
            
        print(Status)
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, Status, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 0.50, color, 2)

    cv2.imshow("My Face", img)
    if cv2.waitKey(10) == 13:    
        break

cv2.destroyAllWindows()

Angry  0.37657958
Neutral  0.25612786
Happy  0.36225834
Happy  0.3929079
Happy  0.43252438
Happy  0.4600886
Happy  0.44554645
Happy  0.44233775
Happy  0.41787386
Happy  0.4454111
Happy  0.4144179
Happy  0.44135517
Happy  0.30163908
Happy  0.36632735
Happy  0.40689567
Happy  0.34360722
Sad  0.42295936
Happy  0.5121586
Happy  0.32462317
Happy  0.3205142
Happy  0.3367198
Happy  0.30839404
Angry  0.31014735
Angry  0.31265515
Angry  0.31970087
